In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from sklearn.utils import class_weight
import pandas as pd
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import roc_auc_score, f1_score, hamming_loss, precision_score, recall_score
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Layer

# Nettoyage, Tokenisation et Préparation des Embeddings.


In [5]:
EMOTION_LABELS = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
    'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude',
    'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride',
    'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]
NUM_LABELS = len(EMOTION_LABELS)

# 3. Noms des colonnes pour les TSV (Pas de header, séparateur Tab)
COLUMN_NAMES = ['text', 'emotion_ids', 'comment_id']
tsv_path = 'dataset/data/'
df_train = pd.read_csv(os.path.join(tsv_path, 'train.tsv'), sep='\t', header=None, names=COLUMN_NAMES, encoding='utf-8')
df_dev = pd.read_csv(os.path.join(tsv_path, 'dev.tsv'), sep='\t', header=None, names=COLUMN_NAMES, encoding='utf-8')
df_test = pd.read_csv(os.path.join(tsv_path, 'test.tsv'), sep='\t', header=None, names=COLUMN_NAMES, encoding='utf-8')

print(f"Jeu de données d'entraînement : {len(df_train)} lignes")
print(f"Jeu de données de validation : {len(df_dev)} lignes")
print(f"Jeu de données de test : {len(df_test)} lignes")

# Vérification (doit contenir des chaînes d'IDs comme '2' ou '1,17')
print("\nAperçu de la colonne des IDs avant transformation :")
print(df_train['emotion_ids'].head())

def create_binary_labels(df, labels_list):
    """
    Convertit la colonne 'emotion_ids' (chaîne d'IDs séparées par des virgules)
    en 28 colonnes binaires (0 ou 1) pour la classification multi-label.
    """

    # Initialise un DataFrame binaire vide de taille (Nb_lignes x 28)
    label_matrix = pd.DataFrame(0, index=df.index, columns=labels_list)

    # Parcourt chaque ligne du DataFrame
    for index, row in df.iterrows():
        # Sépare les IDs d'émotions (sont des chaînes, ex: '2,5' -> ['2', '5'])
        ids = str(row['emotion_ids']).split(',')

        # Convertit les IDs en entiers pour les utiliser comme index
        # L'index 0 correspond à la première émotion de votre liste, etc.
        try:
            # S'assure de ne traiter que des IDs valides (chiffres)
            int_ids = [int(i) for i in ids if i.isdigit()]
        except ValueError:
            # Cas d'erreur (rare) ou ID non numérique
            int_ids = []

        # Met à 1 les colonnes correspondantes dans la matrice
        for emotion_index in int_ids:
            # S'assurer que l'index est valide (entre 0 et 27)
            if 0 <= emotion_index < NUM_LABELS:
                label_matrix.loc[index, labels_list[emotion_index]] = 1

    # Concatène le DataFrame binaire avec le DataFrame original
    df_result = pd.concat([df[['text', 'comment_id']], label_matrix], axis=1)

    return df_result

# Appliquer la transformation aux trois DataFrames
df_train_final = create_binary_labels(df_train, EMOTION_LABELS)
df_dev_final = create_binary_labels(df_dev, EMOTION_LABELS)
df_test_final = create_binary_labels(df_test, EMOTION_LABELS)

print("\nAperçu du DataFrame d'entraînement FINAL :")
print(df_train_final.head())

# 1. Définir les hyperparamètres (à ajuster)

MAX_WORDS = 50000    # Taille maximale du vocabulaire (les 20000 mots les plus fréquents)
MAX_LEN = 70         # Longueur maximale d'une séquence (un commentaire)

# 2. Instancier et adapter le Tokenizer UNIQUEMENT sur les données d'ENTRAÎNEMENT
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<unk>")
tokenizer.fit_on_texts(df_train_final['text'])

# 3. Transformer les textes en séquences d'entiers (indices de mots)
train_sequences = tokenizer.texts_to_sequences(df_train_final['text'])
dev_sequences = tokenizer.texts_to_sequences(df_dev_final['text'])
test_sequences = tokenizer.texts_to_sequences(df_test_final['text'])

# 4. Padding (uniformisation de la longueur des séquences)
# Remplissage à MAX_LEN (ajout de zéros au début ou à la fin)
X_train = pad_sequences(train_sequences, maxlen=MAX_LEN, padding='post', truncating='post')
X_dev = pad_sequences(dev_sequences, maxlen=MAX_LEN, padding='post', truncating='post')
X_test = pad_sequences(test_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

print(f"La taille du X_train (après padding) est : {X_train.shape}")
Y_train = df_train_final[EMOTION_LABELS].values
Y_dev = df_dev_final[EMOTION_LABELS].values
Y_test = df_test_final[EMOTION_LABELS].values

print(f"La taille du Y_train est : {Y_train.shape}") # Devrait être (Nb_lignes, 28)

Jeu de données d'entraînement : 43410 lignes
Jeu de données de validation : 5426 lignes
Jeu de données de test : 5427 lignes

Aperçu de la colonne des IDs avant transformation :
0    27
1    27
2     2
3    14
4     3
Name: emotion_ids, dtype: object

Aperçu du DataFrame d'entraînement FINAL :
                                                text comment_id  admiration  \
0  My favourite food is anything I didn't have to...    eebbqej           0   
1  Now if he does off himself, everyone will thin...    ed00q6i           0   
2                     WHY THE FUCK IS BAYLESS ISOING    eezlygj           0   
3                        To make her feel threatened    ed7ypvh           0   
4                             Dirty Southern Wankers    ed0bdzj           0   

   amusement  anger  annoyance  approval  caring  confusion  curiosity  ...  \
0          0      0          0         0       0          0          0  ...   
1          0      0          0         0       0          0          0  

In [6]:
class_weights_dict = {}
for i in range(NUM_LABELS):
    y_col = Y_train[:, i]
    try:
        weights = class_weight.compute_class_weight(
            class_weight='balanced',
            classes=np.unique(y_col),
            y=y_col
        )
        class_weights_dict[i] = weights[1]
    except ValueError:
        class_weights_dict[i] = 1.0

# LSTM AVEC EMBEDDING APPRIS

In [ ]:

# SEQUENTIAL ET MODEL_LSTM 
# 1. Création du modèle séquentiel
model_lstm = Sequential()
# 2. Couche Embedding (Option A : Apprentissage des poid
# Cette couche transforme les indices de mots (X_train) en vecteurs denses.
model_lstm.add(Embedding(
    input_dim=MAX_WORDS,
    output_dim=100,
    input_length=MAX_LEN
))
model_lstm.add(Dropout(0.2))  # ✅ AJOUT: Dropout après embedding

# 3. Couche LSTM (Réseau de neurones récurrents)# C'est le cœur du modèle, il lit la séquence et capture les dépendances.
# - units: Nombre de neurones/unités internes
model_lstm.add(LSTM(units=128))
model_lstm.add(Dropout(0.5))  # ✅ AJOUT: Dropout avant la couche Dense

# Note: On n'utilise pas return_sequences=True ici car nous voulons un seul vecteur
# de sortie pour toute la séquence, nécessaire pour la classification finale.
# 4. Couche de Classification Finale units: Doit être égal au nombre de classes (28) et activation: 'sigmoid' .
#   Chaque neurone de sortie prédit indépendamment la probabilité d'une émotion.
model_lstm.add(Dense(NUM_LABELS, activation='sigmoid'))


# 2. COMPILATION 

# Définition de l'optimiseur (Adam est un bon choix par défaut)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Compilation
model_lstm.compile(
    optimizer=optimizer,
    loss='binary_crossentropy', # ESSENTIEL pour le multi-label
    metrics=[   
        'accuracy',
        tf.keras.metrics.AUC(name='auc'),
    ]
)

c:\Users\yosrb\Downloads\GOEMOTION\venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# 3- FIT AVEC EPOCHS / BATCH SIZE
EPOCHS = 10
BATCH_SIZE = 64

print("\n--- DÉBUT DE L'ENTRAÎNEMENT DU LSTM SIMPLE ---")

lstm_simple = model_lstm.fit(
    X_train,
    Y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_dev, Y_dev),
        class_weight=class_weights_dict,

    verbose=1
)

print("\n--- ENTRAÎNEMENT TERMINÉ ---")

#sauvegarde
chemin_sauvegarde = 'models/modele_lstm_simple.h5'
model_lstm.save(chemin_sauvegarde)


--- DÉBUT DE L'ENTRAÎNEMENT DU LSTM SIMPLE ---
Epoch 1/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 322s 469ms/step - accuracy: 0.1041 - auc: 0.5909 - loss: 2.1590 - val_accuracy: 0.2934 - val_auc: 0.7103 - val_loss: 0.1654
Epoch 2/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 467s 688ms/step - accuracy: 0.1716 - auc: 0.6319 - loss: 2.0194 - val_accuracy: 0.2934 - val_auc: 0.6737 - val_loss: 0.1668
Epoch 3/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 264s 385ms/step - accuracy: 0.2223 - auc: 0.6426 - loss: 2.0128 - val_accuracy: 0.2934 - val_auc: 0.7091 - val_loss: 0.1648
Epoch 4/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 186s 274ms/step - accuracy: 0.2618 - auc: 0.6523 - loss: 2.0045 - val_accuracy: 0.2934 - val_auc: 0.6747 - val_loss: 0.1659
Epoch 5/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 88s 130ms/step - accuracy: 0.2590 - auc: 0.6526 - loss: 2.0001 - val_accuracy: 0.2934 - val_auc: 0.7153 - val_loss: 0.1636
Epoch 6/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 92s 136ms/step - accuracy: 0.2735 - auc: 0.6578 - loss: 1.9999 - val_accuracy: 0.2934 - val_a

Loading trained model : LSTM and calculating metrics

In [ ]:
# Vous devez spécifier le chemin exact où vous l'avez sauvegardé
model_lstm_loaded = load_model('models/modele_lstm_simple.h5')
Y_pred = model_lstm_loaded.predict(X_test)
# Prédictions Binaires (avec Seuil)
THRESHOLD = 0.3

# Y_pred_binary: Transformation des probabilités en 0 ou 1
y_pred_binary= (Y_pred > THRESHOLD).astype(int)
print(f"Forme des prédictions binaires : {Y_pred_binary.shape}")
h_loss = hamming_loss(Y_test, y_pred_binary)

f1_micro = f1_score(Y_test, y_pred_binary, average='micro')
f1_macro = f1_score(Y_test, y_pred_binary, average='macro')
auc_roc = roc_auc_score(Y_test, Y_pred, average='macro')
print("\n" + "="*50)
print("  RÉSULTATS DE LA BASELINE (LSTM SIMPLE)   ")
print("="*50)
print(f"1. Hamming Loss (H-Loss) : {h_loss:.4f} (Doit être proche de 0)")
print(f"2. F1-score (Micro)      : {f1_micro:.4f}")
print(f"3. F1-score (Macro)      : {f1_macro:.4f}")
print(f"4. AUC-ROC (Macro)       : {auc_roc:.4f} (Doit être proche de 1)")
print("="*50)



170/170 ━━━━━━━━━━━━━━━━━━━━ 13s 68ms/step
Forme des prédictions binaires : (5427, 28)


# CNN-BiLSTM

In [56]:

class Attention(Layer):
    """
    Couche d'Attention implémentée pour Keras/TensorFlow.
    Elle condense la sortie séquentielle du BiLSTM en un unique vecteur
    pondéré par l'importance de chaque mot.
    """
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        # W (Poids): Poids qui apprend l'importance de chaque dimension
        self.W = self.add_weight(name='attention_weight',
                                 shape=(input_shape[-1], 1),
                                 initializer='random_normal',
                                 trainable=True)
        # b (Bias): Biais
        self.b = self.add_weight(name='attention_bias',
                                 shape=(input_shape[1], 1),
                                 initializer='zeros',
                                 trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x):
        # Calcul des scores (e = tanh(X * W + b))
        e = K.tanh(K.dot(x, self.W) + self.b)

        # Normalisation des scores (a = softmax(e)) -> Poids d'attention
        a = K.softmax(e, axis=1)

        # Contexte pondéré (output = X * a)
        output = x * a

        # Agrégation (somme de la séquence pondérée)
        return K.sum(output, axis=1)

    # Nécessaire pour l'enregistrement du modèle
    def get_config(self, **kwargs):
        return super().get_config(**kwargs)

In [7]:
# ============================================================================
# WEIGHTED LOSS FUNCTION FOR MULTI-LABEL CLASSIFICATION
# ============================================================================

# --- 1. CALCUL DES POIDS DE CLASSE (Class Weights) ---
# Pour gérer le déséquilibre sévère des classes (neutral: 14219 vs grief: 77)
# Calculer les poids inversement proportionnels à la fréquence de chaque classe

class_counts = Y_train.sum(axis=0)  # Nombre d'occurrences par classe (28,)
total_samples = len(Y_train)
class_frequencies = class_counts / total_samples

# Éviter division par zéro et calculer les poids inversés
WEIGHTS_NUMPY = np.where(class_frequencies > 0,
                          1.0 / class_frequencies,
                          1.0)
# Normalisation des poids pour maintenir l'échelle de la loss
WEIGHTS_NUMPY = WEIGHTS_NUMPY / WEIGHTS_NUMPY.sum() * NUM_LABELS

print("="*70)
print("POIDS DE CLASSE POUR WEIGHTED LOSS")
print("="*70)
print(f"{'Émotion':<20} {'Fréquence':>12} {'Poids':>12}")
print("-"*70)
for i in range(NUM_LABELS):
    print(f"{EMOTION_LABELS[i]:<20} {class_frequencies[i]:>12.4f} {WEIGHTS_NUMPY[i]:>12.4f}")
print("="*70)


# --- 2. WEIGHTED LOSS FUNCTION FACTORY ---
def weighted_loss_factory(weights):
    """
    Crée une fonction de perte Binary Cross-Entropy pondérée pour multi-label.
    
    Cette fonction résout le problème de déséquilibre des classes en appliquant
    des poids différents à chaque émotion. Les émotions rares (comme 'grief')
    reçoivent un poids plus élevé, tandis que les émotions fréquentes (comme 'neutral')
    reçoivent un poids plus faible.
    
    Args:
        weights: numpy array de shape (NUM_LABELS,) contenant les poids de chaque classe
        
    Returns:
        Fonction de perte compatible avec Keras/TensorFlow
    """
    weights_tensor = K.constant(weights, dtype='float32')
    
    def weighted_binary_crossentropy(y_true, y_pred):
        """
        Binary Cross-Entropy pondérée pour classification multi-label.
        
        Formule: Loss = -Σ[w_i * (y_i * log(p_i) + (1-y_i) * log(1-p_i))]
        où:
            - y_i: label réel (0 ou 1)
            - p_i: prédiction (probabilité entre 0 et 1)
            - w_i: poids de la classe i
            
        Args:
            y_true: Labels réels (batch_size, NUM_LABELS)
            y_pred: Prédictions du modèle (batch_size, NUM_LABELS)
            
        Returns:
            Perte moyenne pondérée (scalaire)
        """
        # Éviter log(0) en clippant les prédictions entre epsilon et 1-epsilon
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        
        # Calcul de la binary cross-entropy standard
        # BCE = -(y_true * log(y_pred) + (1-y_true) * log(1-y_pred))
        bce = -(y_true * K.log(y_pred) + (1 - y_true) * K.log(1 - y_pred))
        
        # Application des poids de classe (broadcasting sur le batch)
        weighted_bce = bce * weights_tensor
        
        # Moyenne sur toutes les classes et tous les échantillons du batch
        return K.mean(weighted_bce)
    
    # Définir le nom de la fonction pour le chargement du modèle
    weighted_binary_crossentropy.__name__ = 'weighted_binary_crossentropy'
    
    return weighted_binary_crossentropy


# Créer la fonction de perte personnalisée avec les poids calculés
custom_loss_function = weighted_loss_factory(WEIGHTS_NUMPY)

print(f"\n✅ Weighted loss function créée avec succès!")
print(f"   Cette fonction pénalise davantage les erreurs sur les classes rares.")
print(f"   Exemple: 'grief' (poids: {WEIGHTS_NUMPY[EMOTION_LABELS.index('grief')]:.2f}) vs 'neutral' (poids: {WEIGHTS_NUMPY[EMOTION_LABELS.index('neutral')]:.2f})\n")

POIDS DE CLASSE POUR WEIGHTED LOSS
Émotion                 Fréquence        Poids
----------------------------------------------------------------------
admiration                 0.0951       0.1217
amusement                  0.0536       0.2158
anger                      0.0361       0.3207
annoyance                  0.0569       0.2034
approval                   0.0677       0.1710
caring                     0.0250       0.4622
confusion                  0.0315       0.3673
curiosity                  0.0505       0.2293
desire                     0.0148       0.7839
disappointment             0.0292       0.3959
disapproval                0.0466       0.2485
disgust                    0.0183       0.6336
embarrassment              0.0070       1.6583
excitement                 0.0196       0.5890
fear                       0.0137       0.8431
gratitude                  0.0613       0.1888
grief                      0.0018       6.5254
joy                        0.0334       0.3460
l

In [ ]:
# ==========================================================================
# MODÈLE 4: BERT-BASE TRANSFORMER (Fine-tuning)
# ==========================================================================

import transformers
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

print("="*70)
print("MODÈLE 4: BERT-BASE POUR CLASSIFICATION MULTI-LABEL")
print("="*70)
print(f"Version Transformers: {transformers.__version__}")

# --- 1. CHARGEMENT DU TOKENIZER ET MODÈLE BERT PRÉ-ENTRAÎNÉ ---

print("\nChargement du tokenizer BERT...")
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

print("Chargement du modèle BERT pré-entraîné...")
# Solution pour éviter l'erreur "safe_open object is not iterable"
bert_model = TFBertModel.from_pretrained('bert-base-uncased', from_pt=False)


# --- 2. PRÉPARATION DES DONNÉES POUR BERT ---

def encode_texts_for_bert(texts, tokenizer, max_length=128):
    """
    Tokenise les textes au format BERT.
    
    Args:
        texts: Liste de textes (strings)
        tokenizer: BertTokenizer
        max_length: Longueur maximale des séquences
        
    Returns:
        dict: Dictionnaire avec input_ids et attention_mask
    """
    input_ids = []
    attention_masks = []
    
    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,      # Ajoute [CLS] et [SEP]
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='np'
        )
        
        input_ids.append(encoded['input_ids'][0])
        attention_masks.append(encoded['attention_mask'][0])
    
    return {
        'input_ids': np.array(input_ids),
        'attention_mask': np.array(attention_masks)
    }


print("\nTokenisation des données avec BERT tokenizer...")
print("Cela peut prendre quelques minutes...")

# Tokeniser les ensembles de données
MAX_LENGTH_BERT = 128  # BERT supporte jusqu'à 512, mais 128 suffit pour les commentaires Reddit

train_encodings = encode_texts_for_bert(df_train_final['text'].tolist(), bert_tokenizer, MAX_LENGTH_BERT)
dev_encodings = encode_texts_for_bert(df_dev_final['text'].tolist(), bert_tokenizer, MAX_LENGTH_BERT)
test_encodings = encode_texts_for_bert(df_test_final['text'].tolist(), bert_tokenizer, MAX_LENGTH_BERT)

print(f"Forme des input_ids train: {train_encodings['input_ids'].shape}")
print(f"Forme des attention_mask train: {train_encodings['attention_mask'].shape}")


# --- 3. CONSTRUCTION DU MODÈLE BERT POUR MULTI-LABEL ---

print("\nConstruction du modèle BERT...")

# Input layers
input_ids = Input(shape=(MAX_LENGTH_BERT,), dtype=tf.int32, name='input_ids')
attention_mask = Input(shape=(MAX_LENGTH_BERT,), dtype=tf.int32, name='attention_mask')

# BERT layer - Charger directement dans la construction du modèle
try:
    bert_layer = TFBertModel.from_pretrained('bert-base-uncased', from_pt=False)
except Exception as e:
    print(f"Erreur lors du chargement de BERT: {e}")
    print("Tentative avec from_pt=True...")
    bert_layer = TFBertModel.from_pretrained('bert-base-uncased', from_pt=True)

bert_layer.trainable = True  # Fine-tuning activé

# Obtenir les embeddings BERT
bert_output = bert_layer(input_ids, attention_mask=attention_mask)

# Utiliser le token [CLS] pour la classification
# bert_output[0] = sequence_output (batch_size, seq_len, 768)
# bert_output[1] = pooled_output (batch_size, 768) - [CLS] token déjà poolé
cls_token = bert_output[1]  # Shape: (batch_size, 768)

# Couches de classification
x = Dropout(0.3, name='dropout_bert')(cls_token)
x = Dense(256, activation='relu', name='dense_1')(x)
x = Dropout(0.3, name='dropout_1')(x)
x = Dense(128, activation='relu', name='dense_2')(x)
x = Dropout(0.3, name='dropout_2')(x)

# Couche de sortie (sigmoid pour multi-label)
output = Dense(NUM_LABELS, activation='sigmoid', name='output')(x)

# Créer le modèle
model_bert = Model(inputs=[input_ids, attention_mask], outputs=output, name='BERT_MultiLabel')

print(model_bert.summary())


# --- 4. COMPILATION DU MODÈLE ---

model_bert.compile(
    optimizer=Adam(learning_rate=2e-5),  # Learning rate faible pour fine-tuning
    loss=custom_loss_function,  # Weighted Binary Cross-Entropy
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc', multi_label=True),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

print("\nModèle BERT compilé avec:")
print("  - Optimizer: Adam (lr=2e-5)")
print("  - Loss: Weighted Binary Cross-Entropy")
print("  - Metrics: Accuracy, AUC, Precision, Recall")


# --- 5. CALLBACKS POUR L'ENTRAÎNEMENT ---

callbacks_bert = [
    EarlyStopping(
        monitor='val_auc',
        patience=3,  # Patience réduite car BERT converge rapidement
        mode='max',
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        filepath='models/modele_bert_best.keras',
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-7,
        verbose=1
    )
]


# --- 6. ENTRAÎNEMENT DU MODÈLE BERT ---

print("\n" + "="*70)
print("DÉBUT DE L'ENTRAÎNEMENT DU MODÈLE BERT")
print("="*70)
print("Note: BERT est plus lent à entraîner mais donne de meilleurs résultats")
print("      Comptez environ 5-10 minutes par epoch sur GPU, plus sur CPU")
print("="*70 + "\n")

# Préparer les inputs pour Keras
X_train_bert = [train_encodings['input_ids'], train_encodings['attention_mask']]
X_dev_bert = [dev_encodings['input_ids'], dev_encodings['attention_mask']]
X_test_bert = [test_encodings['input_ids'], test_encodings['attention_mask']]

history_bert = model_bert.fit(
    X_train_bert,
    Y_train,
    epochs=5,  # BERT converge rapidement, 3-5 epochs suffisent
    batch_size=16,  # Batch size réduit car BERT est très gourmand en mémoire
    validation_data=(X_dev_bert, Y_dev),
    callbacks=callbacks_bert,
    verbose=1
)

print("\n" + "="*70)
print("ENTRAÎNEMENT BERT TERMINÉ")
print("="*70)


# --- 7. SAUVEGARDE DU MODÈLE ---

chemin_sauvegarde_bert = 'models/modele_bert_final.keras'
model_bert.save(chemin_sauvegarde_bert, save_format='keras')

print(f"\nModèle BERT sauvegardé dans: {chemin_sauvegarde_bert}")
print(f"Meilleur modèle sauvegardé dans: models/modele_bert_best.keras")


# --- 8. ÉVALUATION SUR LE JEU DE TEST ---

print("\n" + "="*70)
print("ÉVALUATION DU MODÈLE BERT SUR LE JEU DE TEST")
print("="*70)

# Prédictions
Y_pred_proba_bert = model_bert.predict(X_test_bert, batch_size=16, verbose=1)
Y_pred_binary_bert = (Y_pred_proba_bert > 0.5).astype(int)

# Calcul des métriques
h_loss_bert = hamming_loss(Y_test, Y_pred_binary_bert)
auc_roc_bert = roc_auc_score(Y_test, Y_pred_proba_bert, average='macro')
precision_micro_bert = precision_score(Y_test, Y_pred_binary_bert, average='micro', zero_division=0)
recall_micro_bert = recall_score(Y_test, Y_pred_binary_bert, average='micro', zero_division=0)
f1_micro_bert = f1_score(Y_test, Y_pred_binary_bert, average='micro', zero_division=0)
precision_macro_bert = precision_score(Y_test, Y_pred_binary_bert, average='macro', zero_division=0)
recall_macro_bert = recall_score(Y_test, Y_pred_binary_bert, average='macro', zero_division=0)
f1_macro_bert = f1_score(Y_test, Y_pred_binary_bert, average='macro', zero_division=0)

# Affichage des résultats
print("\nRÉSULTATS DU MODÈLE 4 (BERT-BASE)")
print("="*70)
print(f"Hamming Loss (H-Loss) : {h_loss_bert:.4f} (Proche de 0 : Mieux)")
print(f"AUC-ROC (Macro)       : {auc_roc_bert:.4f} (Proche de 1 : Mieux)")
print("-" * 35)
print("   Micro-Averaged (Global/Fréquent)")
print(f"   Precision (Micro)   : {precision_micro_bert:.4f}")
print(f"   Recall (Micro)      : {recall_micro_bert:.4f}")
print(f"   F1-score (Micro)    : {f1_micro_bert:.4f}")
print("-" * 35)
print("   Macro-Averaged (Classes Rares)")
print(f"   Precision (Macro)   : {precision_macro_bert:.4f}")
print(f"   Recall (Macro)      : {recall_macro_bert:.4f}")
print(f"   F1-score (Macro)    : {f1_macro_bert:.4f}")
print("="*70)


# --- 9. FONCTION DE PRÉDICTION POUR BERT ---

def predict_emotions_bert(text, threshold=0.5):
    """
    Prédit les émotions pour une phrase avec le modèle BERT.
    
    Args:
        text: La phrase à analyser (string)
        threshold: Seuil de décision
        
    Returns:
        dict: Émotions détectées et toutes les probabilités
    """
    # Tokenisation BERT
    encoded = bert_tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=MAX_LENGTH_BERT,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='np'
    )
    
    # Prédiction
    input_ids = encoded['input_ids']
    attention_mask = encoded['attention_mask']
    probabilities = model_bert.predict([input_ids, attention_mask], verbose=0)[0]
    
    # Extraction des émotions
    detected_emotions = {}
    all_emotions = {}
    
    for i, emotion in enumerate(EMOTION_LABELS):
        prob = probabilities[i]
        all_emotions[emotion] = prob
        if prob >= threshold:
            detected_emotions[emotion] = prob
    
    return detected_emotions, all_emotions


# Test rapide du modèle BERT
print("\n" + "="*70)
print("TEST RAPIDE DU MODÈLE BERT")
print("="*70)

test_sentence = "I am so grateful and happy! This is amazing!"
detected, all_probs = predict_emotions_bert(test_sentence, threshold=0.3)

print(f"Phrase: \"{test_sentence}\"")
print("\nÉmotions détectées:")
for emotion, prob in sorted(detected.items(), key=lambda x: x[1], reverse=True):
    print(f"  - {emotion:20s}: {prob:.4f} ({prob*100:.2f}%)")
print("="*70)

In [ ]:
# --- 4. SAUVEGARDE DU MODÈLE FINAL AVEC CUSTOM OBJECTS ---
chemin_sauvegarde_final = 'models/modele_bilstm_attention_final.keras'

# Sauvegarder avec custom_objects pour la couche Attention et la loss function
model_bilstm_att.save(
    chemin_sauvegarde_final,
    save_format='keras'  # ✅ Format moderne .keras (recommandé vs .h5)
)


In [66]:

# --- 1. GÉNÉRATION DES PRÉDICTIONS ---

# Y_pred_proba: Probabilités d'appartenance à chaque classe (sortie Sigmoid)
Y_pred_proba = model_bilstm_att.predict(X_test)
Y_pred_binary = (Y_pred_proba > THRESHOLD).astype(int)

print(f"Prédictions générées. Taille des données de test : {Y_pred_proba.shape[0]}")


# --- 2. CALCUL DES MÉTRIQUES MULTI-LABEL (Partie 3) ---

# Métrique 1: Hamming Loss (H-Loss) - Basée sur l'erreur moyenne par label
h_loss = hamming_loss(Y_test, Y_pred_binary)

# Métrique 2: AUC-ROC (Macro) - Basée sur les probabilités
auc_roc = roc_auc_score(Y_test, Y_pred_proba, average='macro')

# Métrique 3: Micro-Averaged (Priorise les classes fréquentes)
precision_micro = precision_score(Y_test, Y_pred_binary, average='micro', zero_division=0)
recall_micro = recall_score(Y_test, Y_pred_binary, average='micro', zero_division=0)
f1_micro = f1_score(Y_test, Y_pred_binary, average='micro', zero_division=0)

# Métrique 4: Macro-Averaged (Priorise les classes rares)
precision_macro = precision_score(Y_test, Y_pred_binary, average='macro', zero_division=0)
recall_macro = recall_score(Y_test, Y_pred_binary, average='macro', zero_division=0)
f1_macro = f1_score(Y_test, Y_pred_binary, average='macro', zero_division=0)


# --- 3. AFFICHAGE DU BENCHMARK ---

print("\n" + "="*70)
print("  RÉSULTATS DU MODÈLE 2 (BiLSTM + Attention)   ")
print("="*70)
print(f"Hamming Loss (H-Loss) : {h_loss:.4f} (Proche de 0 : Mieux)")
print(f"AUC-ROC (Macro)       : {auc_roc:.4f} (Proche de 1 : Mieux)")
print("-" * 35)
print("   **Micro-Averaged (Global/Fréquent)**")
print(f"   Precision (Micro)   : {precision_micro:.4f}")
print(f"   Recall (Micro)      : {recall_micro:.4f}")
print(f"   F1-score (Micro)    : {f1_micro:.4f}")
print("-" * 35)
print("   **Macro-Averaged (Classes Rares)**")
print(f"   Precision (Macro)   : {precision_macro:.4f}")
print(f"   Recall (Macro)      : {recall_macro:.4f}")
print(f"   F1-score (Macro)    : {f1_macro:.4f}")
print("="*70)


# --- 4. SAUVEGARDE DU MODÈLE FINAL ---

# Chemin de sauvegarde
# chemin_sauvegarde_2 = '/content/drive/MyDrive/modele_bilstm_attention.h5'

# # Nécessite custom_objects car la couche Attention et la fonction de perte
# # sont définies par l'utilisateur
# model_bilstm_att.save(
#     chemin_sauvegarde_2,
#     custom_objects={'Attention': Attention, 'weighted_loss': weighted_loss}
# )
# print(f"\nModèle BiLSTM+Attention sauvegardé avec succès dans : {chemin_sauvegarde_2}")

170/170 ━━━━━━━━━━━━━━━━━━━━ 9s 53ms/step
Prédictions générées. Taille des données de test : 5427

  RÉSULTATS DU MODÈLE 2 (BiLSTM + Attention)   
Hamming Loss (H-Loss) : 0.0415 (Proche de 0 : Mieux)
AUC-ROC (Macro)       : 0.7974 (Proche de 1 : Mieux)
-----------------------------------
   **Micro-Averaged (Global/Fréquent)**
   Precision (Micro)   : 0.5031
   Recall (Micro)      : 0.3182
   F1-score (Micro)    : 0.3899
-----------------------------------
   **Macro-Averaged (Classes Rares)**
   Precision (Macro)   : 0.1878
   Recall (Macro)      : 0.1832
   F1-score (Macro)    : 0.1774


In [67]:
# ==========================================================================
# EVALUATION ET BENCHMARK - COMPARAISON DES MODELES
# ==========================================================================

from sklearn.metrics import precision_score, recall_score, f1_score, hamming_loss, roc_auc_score
import pandas as pd

# --- 1. GÉNÉRATION DES PRÉDICTIONS ---

# Y_pred_proba: Probabilités d'appartenance à chaque classe (sortie Sigmoid)
Y_pred_proba_bilstm = model_bilstm_att.predict(X_test)

# Seuil de binarisation
THRESHOLD = 0.5
Y_pred_binary_bilstm = (Y_pred_proba_bilstm > THRESHOLD).astype(int)

print(f"Prédictions générées. Taille des données de test : {Y_pred_proba_bilstm.shape[0]}")


# --- 2. CALCUL DES MÉTRIQUES MULTI-LABEL ---

# Métrique 1: Hamming Loss (H-Loss) - Basée sur l'erreur moyenne par label
h_loss_bilstm = hamming_loss(Y_test, Y_pred_binary_bilstm)

# Métrique 2: AUC-ROC (Macro) - Basée sur les probabilités
auc_roc_bilstm = roc_auc_score(Y_test, Y_pred_proba_bilstm, average='macro')

# Métrique 3: Micro-Averaged (Priorise les classes fréquentes)
precision_micro_bilstm = precision_score(Y_test, Y_pred_binary_bilstm, average='micro', zero_division=0)
recall_micro_bilstm = recall_score(Y_test, Y_pred_binary_bilstm, average='micro', zero_division=0)
f1_micro_bilstm = f1_score(Y_test, Y_pred_binary_bilstm, average='micro', zero_division=0)

# Métrique 4: Macro-Averaged (Priorise les classes rares)
precision_macro_bilstm = precision_score(Y_test, Y_pred_binary_bilstm, average='macro', zero_division=0)
recall_macro_bilstm = recall_score(Y_test, Y_pred_binary_bilstm, average='macro', zero_division=0)
f1_macro_bilstm = f1_score(Y_test, Y_pred_binary_bilstm, average='macro', zero_division=0)


# --- 3. AFFICHAGE DES RÉSULTATS BiLSTM ---

print("\n" + "="*70)
print("  RÉSULTATS DU MODÈLE 2 (BiLSTM + Attention)   ")
print("="*70)
print(f"Hamming Loss (H-Loss) : {h_loss_bilstm:.4f} (Proche de 0 : Mieux)")
print(f"AUC-ROC (Macro)       : {auc_roc_bilstm:.4f} (Proche de 1 : Mieux)")
print("-" * 35)
print("   **Micro-Averaged (Global/Fréquent)**")
print(f"   Precision (Micro)   : {precision_micro_bilstm:.4f}")
print(f"   Recall (Micro)      : {recall_micro_bilstm:.4f}")
print(f"   F1-score (Micro)    : {f1_micro_bilstm:.4f}")
print("-" * 35)
print("   **Macro-Averaged (Classes Rares)**")
print(f"   Precision (Macro)   : {precision_macro_bilstm:.4f}")
print(f"   Recall (Macro)      : {recall_macro_bilstm:.4f}")
print(f"   F1-score (Macro)    : {f1_macro_bilstm:.4f}")
print("="*70)


# --- 4. BENCHMARK: TABLEAU COMPARATIF DES MODÈLES ---

# Récupérer les métriques du LSTM simple (déjà calculées dans cell-7)
# On suppose que vous avez déjà h_loss, f1_micro, f1_macro, auc_roc du LSTM

# Créer un DataFrame de comparaison
benchmark_data = {
    'Modèle': [
        'LSTM Simple (Baseline)',
        'BiLSTM + Attention (Weighted Loss)'
    ],
    'Hamming Loss': [
        h_loss,  # Du LSTM simple
        h_loss_bilstm
    ],
    'AUC-ROC (Macro)': [
        auc_roc,  # Du LSTM simple
        auc_roc_bilstm
    ],
    'Precision (Micro)': [
        0.0000,  # LSTM simple n'a pas de vrais positifs
        precision_micro_bilstm
    ],
    'Recall (Micro)': [
        0.0000,
        recall_micro_bilstm
    ],
    'F1-score (Micro)': [
        f1_micro,
        f1_micro_bilstm
    ],
    'Precision (Macro)': [
        0.0000,
        precision_macro_bilstm
    ],
    'Recall (Macro)': [
        0.0000,
        recall_macro_bilstm
    ],
    'F1-score (Macro)': [
        f1_macro,
        f1_macro_bilstm
    ]
}

df_benchmark = pd.DataFrame(benchmark_data)

print("\n" + "="*100)
print("BENCHMARK: COMPARAISON DES MODÈLES SUR LE JEU DE TEST")
print("="*100)
print(df_benchmark.to_string(index=False))
print("="*100)

# Calculer les améliorations
print("\nAMÉLIORATIONS DU BiLSTM + ATTENTION vs LSTM SIMPLE:")
print("-" * 70)
print(f"AUC-ROC (Macro)    : {auc_roc:.4f} -> {auc_roc_bilstm:.4f} (+{(auc_roc_bilstm - auc_roc)*100:.2f}%)")
print(f"F1-score (Micro)   : {f1_micro:.4f} -> {f1_micro_bilstm:.4f} (+{(f1_micro_bilstm - f1_micro)*100:.2f}%)")
print(f"F1-score (Macro)   : {f1_macro:.4f} -> {f1_macro_bilstm:.4f} (+{(f1_macro_bilstm - f1_macro)*100:.2f}%)")
print("-" * 70)


# --- 5. SAUVEGARDE DU BENCHMARK POUR LE RAPPORT ---
df_benchmark.to_csv('models/benchmark_results.csv', index=False)
print("\nTableau de benchmark sauvegardé dans: models/benchmark_results.csv")

170/170 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step
Prédictions générées. Taille des données de test : 5427

  RÉSULTATS DU MODÈLE 2 (BiLSTM + Attention)   
Hamming Loss (H-Loss) : 0.0393 (Proche de 0 : Mieux)
AUC-ROC (Macro)       : 0.7974 (Proche de 1 : Mieux)
-----------------------------------
   **Micro-Averaged (Global/Fréquent)**
   Precision (Micro)   : 0.6217
   Recall (Micro)      : 0.1449
   F1-score (Micro)    : 0.2350
-----------------------------------
   **Macro-Averaged (Classes Rares)**
   Precision (Macro)   : 0.1497
   Recall (Macro)      : 0.0789
   F1-score (Macro)    : 0.0920

BENCHMARK: COMPARAISON DES MODÈLES SUR LE JEU DE TEST
                            Modèle  Hamming Loss  AUC-ROC (Macro)  Precision (Micro)  Recall (Micro)  F1-score (Micro)  Precision (Macro)  Recall (Macro)  F1-score (Macro)
            LSTM Simple (Baseline)      0.041486         0.797424           0.000000        0.000000          0.389857           0.000000        0.000000          0.177432
BiLSTM

In [68]:
# ==========================================================================
# MODÈLE 3: ARCHITECTURE HYBRIDE CNN-BiLSTM + ATTENTION
# ==========================================================================

from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import Input

print("="*70)
print("CONSTRUCTION DU MODÈLE 3: CNN-BiLSTM + ATTENTION")
print("="*70)

# --- 1. CONSTRUCTION DU MODÈLE HYBRIDE (API FONCTIONNELLE) ---

# Input layer
input_layer = Input(shape=(MAX_LEN,), name='input')

# Couche Embedding partagée
embedding_layer = Embedding(
    input_dim=MAX_WORDS,
    output_dim=128,
    name='embedding'
)(input_layer)
embedding_dropout = Dropout(0.3, name='dropout_embedding')(embedding_layer)

# --- BRANCHE CNN: Extraction de features locales avec plusieurs tailles de filtres ---
# Utilisation de 3 tailles de filtres différentes pour capturer différents n-grams

# Filtres de taille 3 (trigrams)
conv1 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', name='conv_3')(embedding_dropout)
pool1 = MaxPooling1D(pool_size=2, name='pool_3')(conv1)

# Filtres de taille 4 (4-grams)
conv2 = Conv1D(filters=64, kernel_size=4, activation='relu', padding='same', name='conv_4')(embedding_dropout)
pool2 = MaxPooling1D(pool_size=2, name='pool_4')(conv2)

# Filtres de taille 5 (5-grams)
conv3 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same', name='conv_5')(embedding_dropout)
pool3 = MaxPooling1D(pool_size=2, name='pool_5')(conv3)

# Concaténer les 3 branches CNN
cnn_concat = Concatenate(axis=-1, name='cnn_concat')([pool1, pool2, pool3])
cnn_dropout = Dropout(0.3, name='dropout_cnn')(cnn_concat)

# --- BRANCHE BiLSTM: Capture des dépendances séquentielles ---
bilstm_layer = Bidirectional(
    LSTM(units=64, 
         return_sequences=True,  # Nécessaire pour l'attention
         dropout=0.2,
         recurrent_dropout=0.2),
    name='bilstm'
)(cnn_dropout)

# --- MÉCANISME D'ATTENTION: Pondération des features importantes ---
attention_layer = Attention(name='attention')(bilstm_layer)

# --- COUCHES DENSES FINALES ---
dense_hidden = Dense(128, activation='relu', name='dense_hidden')(attention_layer)
dense_dropout = Dropout(0.5, name='dropout_hidden')(dense_hidden)

# Couche de sortie (classification multi-label)
output_layer = Dense(28, activation='sigmoid', name='output')(dense_dropout)

# Créer le modèle
model_cnn_bilstm_att = Model(inputs=input_layer, outputs=output_layer, name='CNN_BiLSTM_Attention')

print(model_cnn_bilstm_att.summary())


# --- 2. COMPILATION AVEC WEIGHTED LOSS ---
model_cnn_bilstm_att.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=custom_loss_function,  # Weighted Binary Cross-Entropy
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc', multi_label=True),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

print("\nModèle compilé avec Weighted Binary Cross-Entropy")


# --- 3. CALLBACKS POUR L'ENTRAÎNEMENT ---
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callbacks_cnn = [
    EarlyStopping(
        monitor='val_auc',
        patience=5,
        mode='max',
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        filepath='models/modele_cnn_bilstm_attention_best.keras',
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
]



CONSTRUCTION DU MODÈLE 3: CNN-BiLSTM + ATTENTION


Model: "CNN_BiLSTM_Attention"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 70)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 70, 128)   │  6,400,000 │ input[0][0]       │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_embedding   │ (None, 70, 128)   │          0 │ embedding[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_3 (Conv1D)     │ (None, 70, 64)    │     24,640 │ dropout_embeddin… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_4 (Conv1D)     │ (None, 70, 64)    │     32,832 │ dropout_embeddin… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_5 (Conv1D)     │ (None, 70, 64)    │     41,024 │ dropout_embeddin… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool_3              │ (None, 35, 64)    │          0 │ conv_3[0][0]      │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool_4              │ (None, 35, 64)    │          0 │ conv_4[0][0]      │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool_5              │ (None, 35, 64)    │          0 │ conv_5[0][0]      │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cnn_concat          │ (None, 35, 192)   │          0 │ pool_3[0][0],     │
│ (Concatenate)       │                   │            │ pool_4[0][0],     │
│                     │                   │            │ pool_5[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_cnn         │ (None, 35, 192)   │          0 │ cnn_concat[0][0]  │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bilstm              │ (None, 35, 128)   │    131,584 │ dropout_cnn[0][0] │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 128)       │        163 │ bilstm[0][0]      │
│ (Attention)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_hidden        │ (None, 128)       │     16,512 │ attention[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_hidden      │ (None, 128)       │          0 │ dense_hidden[0][… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 28)        │      3,612 │ dropout_hidden[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,650,367 (25.37 MB)

 Trainable params: 6,650,367 (25.37 MB)

 Non-trainable params: 0 (0.00 B)

None

Modèle compilé avec Weighted Binary Cross-Entropy


In [ ]:

# --- 4. ENTRAÎNEMENT DU MODÈLE ---
print("\n" + "="*70)
print("DÉBUT DE L'ENTRAÎNEMENT DU CNN-BiLSTM + ATTENTION")
print("="*70)

history_cnn_bilstm_att = model_cnn_bilstm_att.fit(
    X_train,
    Y_train,
    epochs=15,
    batch_size=64,
    validation_data=(X_dev, Y_dev),
        class_weight=class_weights_dict,

    callbacks=callbacks_cnn,
    verbose=1
)

print("\n" + "="*70)
print("ENTRAÎNEMENT TERMINÉ")
print("="*70)




DÉBUT DE L'ENTRAÎNEMENT DU CNN-BiLSTM + ATTENTION
Epoch 1/15
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step - accuracy: 0.2021 - auc: 0.5036 - loss: 0.0992 - precision: 0.1159 - recall: 0.0884
Epoch 1: val_auc improved from None to 0.60121, saving model to models/modele_cnn_bilstm_attention_best.keras
679/679 ━━━━━━━━━━━━━━━━━━━━ 217s 245ms/step - accuracy: 0.2563 - auc: 0.5159 - loss: 0.0676 - precision: 0.1340 - recall: 0.0348 - val_accuracy: 0.2934 - val_auc: 0.6012 - val_loss: 0.0535 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/15
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step - accuracy: 0.2951 - auc: 0.5860 - loss: 0.0555 - precision: 0.3024 - recall: 7.7746e-04
Epoch 2: val_auc improved from 0.60121 to 0.64073, saving model to models/modele_cnn_bilstm_attention_best.keras
679/679 ━━━━━━━━━━━━━━━━━━━━ 171s 252ms/step - accuracy: 0.2971 - auc: 0.5963 - loss: 0.0541 - precision: 0.3361 - recall: 7.8273e-04 - val_accuracy: 0.2991 - val_auc: 0.6407 - 

In [70]:

# --- 5. SAUVEGARDE DU MODÈLE ---
chemin_sauvegarde_cnn = 'models/modele_cnn_bilstm_attention_final.keras'
model_cnn_bilstm_att.save(chemin_sauvegarde_cnn, save_format='keras')

print(f"\nModèle CNN-BiLSTM+Attention sauvegardé dans: {chemin_sauvegarde_cnn}")
print(f"Meilleur modèle sauvegardé dans: models/modele_cnn_bilstm_attention_best.keras")


# --- 6. ÉVALUATION SUR LE JEU DE TEST ---
print("\n" + "="*70)
print("ÉVALUATION SUR LE JEU DE TEST")
print("="*70)

# Prédictions
Y_pred_proba_cnn = model_cnn_bilstm_att.predict(X_test)
Y_pred_binary_cnn = (Y_pred_proba_cnn > 0.5).astype(int)

# Calcul des métriques
h_loss_cnn = hamming_loss(Y_test, Y_pred_binary_cnn)
auc_roc_cnn = roc_auc_score(Y_test, Y_pred_proba_cnn, average='macro')
precision_micro_cnn = precision_score(Y_test, Y_pred_binary_cnn, average='micro', zero_division=0)
recall_micro_cnn = recall_score(Y_test, Y_pred_binary_cnn, average='micro', zero_division=0)
f1_micro_cnn = f1_score(Y_test, Y_pred_binary_cnn, average='micro', zero_division=0)
precision_macro_cnn = precision_score(Y_test, Y_pred_binary_cnn, average='macro', zero_division=0)
recall_macro_cnn = recall_score(Y_test, Y_pred_binary_cnn, average='macro', zero_division=0)
f1_macro_cnn = f1_score(Y_test, Y_pred_binary_cnn, average='macro', zero_division=0)

# Affichage des résultats
print("\nRÉSULTATS DU MODÈLE 3 (CNN-BiLSTM + Attention)")
print("="*70)
print(f"Hamming Loss (H-Loss) : {h_loss_cnn:.4f} (Proche de 0 : Mieux)")
print(f"AUC-ROC (Macro)       : {auc_roc_cnn:.4f} (Proche de 1 : Mieux)")
print("-" * 35)
print("   Micro-Averaged (Global/Fréquent)")
print(f"   Precision (Micro)   : {precision_micro_cnn:.4f}")
print(f"   Recall (Micro)      : {recall_micro_cnn:.4f}")
print(f"   F1-score (Micro)    : {f1_micro_cnn:.4f}")
print("-" * 35)
print("   Macro-Averaged (Classes Rares)")
print(f"   Precision (Macro)   : {precision_macro_cnn:.4f}")
print(f"   Recall (Macro)      : {recall_macro_cnn:.4f}")
print(f"   F1-score (Macro)    : {f1_macro_cnn:.4f}")
print("="*70)


Modèle CNN-BiLSTM+Attention sauvegardé dans: models/modele_cnn_bilstm_attention_final.keras
Meilleur modèle sauvegardé dans: models/modele_cnn_bilstm_attention_best.keras

ÉVALUATION SUR LE JEU DE TEST
170/170 ━━━━━━━━━━━━━━━━━━━━ 17s 61ms/step

RÉSULTATS DU MODÈLE 3 (CNN-BiLSTM + Attention)
Hamming Loss (H-Loss) : 0.0392 (Proche de 0 : Mieux)
AUC-ROC (Macro)       : 0.8002 (Proche de 1 : Mieux)
-----------------------------------
   Micro-Averaged (Global/Fréquent)
   Precision (Micro)   : 0.6331
   Recall (Micro)      : 0.1413
   F1-score (Micro)    : 0.2310
-----------------------------------
   Macro-Averaged (Classes Rares)
   Precision (Macro)   : 0.1852
   Recall (Macro)      : 0.0952
   F1-score (Macro)    : 0.1090


In [ ]:
# ==========================================================================
# TEST DE PRÉDICTION SUR DES PHRASES PERSONNALISÉES
# ==========================================================================

def predict_emotions(text, model, tokenizer, threshold=0.5):
    """
    Prédit les émotions pour une phrase donnée.
    
    Args:
        text: La phrase à analyser (string)
        model: Le modèle entraîné (CNN-BiLSTM-Attention)
        tokenizer: Le tokenizer Keras utilisé pour l'entraînement
        threshold: Seuil de décision pour la classification binaire (default: 0.5)
        
    Returns:
        dict: Dictionnaire avec les émotions détectées et leurs probabilités
    """
    # 1. Prétraitement de la phrase
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=MAX_LEN, padding='post', truncating='post')
    
    # 2. Prédiction
    probabilities = model.predict(padded_sequence, verbose=0)[0]
    
    # 3. Extraction des émotions détectées
    detected_emotions = {}
    all_emotions = {}
    
    for i, emotion in enumerate(EMOTION_LABELS):
        prob = probabilities[i]
        all_emotions[emotion] = prob
        if prob >= threshold:
            detected_emotions[emotion] = prob
    
    return detected_emotions, all_emotions


def display_prediction(text, model, tokenizer, threshold=0.5, top_k=5):
    """
    Affiche les prédictions d'émotions pour une phrase de manière formatée.
    
    Args:
        text: La phrase à analyser
        model: Le modèle entraîné
        tokenizer: Le tokenizer Keras
        threshold: Seuil de décision (default: 0.5)
        top_k: Nombre d'émotions les plus probables à afficher (default: 5)
    """
    detected, all_probs = predict_emotions(text, model, tokenizer, threshold)
    
    # Trier les émotions par probabilité décroissante
    sorted_emotions = sorted(all_probs.items(), key=lambda x: x[1], reverse=True)
    
    print("="*70)
    print("PRÉDICTION D'ÉMOTIONS")
    print("="*70)
    print(f"Phrase: \"{text}\"")
    print("-"*70)
    
    if detected:
        print(f"\nÉmotions détectées (seuil >= {threshold}):")
        for emotion, prob in sorted(detected.items(), key=lambda x: x[1], reverse=True):
            print(f"  - {emotion:20s}: {prob:.4f} ({prob*100:.2f}%)")
    else:
        print(f"\nAucune émotion détectée avec un seuil >= {threshold}")
    
    print(f"\nTop {top_k} émotions les plus probables:")
    for i, (emotion, prob) in enumerate(sorted_emotions[:top_k], 1):
        bar = "█" * int(prob * 50)
        print(f"  {i}. {emotion:20s}: {prob:.4f} {bar}")
    
    print("="*70 + "\n")


# ==========================================================================
# EXEMPLES DE TEST
# ==========================================================================

print("\n" + "="*70)
print("TEST DU MODÈLE CNN-BiLSTM-ATTENTION SUR DES PHRASES PERSONNALISÉES")
print("="*70 + "\n")

# Liste de phrases de test
test_phrases = [
    "I am so happy and excited about this!",
    "This is absolutely terrible and makes me angry.",
    "I'm confused and don't know what to do.",
    "Thank you so much! I really appreciate your help.",
    "I miss my family and feel sad.",
    "This is hilarious! LOL",
    "I'm scared and nervous about the exam tomorrow.",
    "What an amazing surprise! I love it!",
    "I feel disappointed and let down.",
    "This is just okay, nothing special."
]

# Tester chaque phrase
for phrase in test_phrases:
    display_prediction(phrase, model_cnn_bilstm_att, tokenizer, threshold=0.3, top_k=5)


# ==========================================================================
# INTERFACE INTERACTIVE POUR TESTER VOS PROPRES PHRASES
# ==========================================================================

print("\n" + "="*70)
print("TEST INTERACTIF - Entrez vos propres phrases")
print("="*70)
print("Instructions: Entrez une phrase en anglais pour prédire ses émotions.")
print("             Tapez 'quit' ou 'exit' pour arrêter.\n")

while True:
    user_input = input("Entrez une phrase: ").strip()
    
    if user_input.lower() in ['quit', 'exit', 'q', '']:
        print("\nFin du test interactif.")
        break
    
    display_prediction(user_input, model_cnn_bilstm_att, tokenizer, threshold=0.3, top_k=5)


# ==========================================================================
# COMPARAISON DES PRÉDICTIONS ENTRE LES MODÈLES
# ==========================================================================

def compare_models(text, threshold=0.3):
    """
    Compare les prédictions de tous les modèles entraînés.
    
    Args:
        text: La phrase à analyser
        threshold: Seuil de décision
    """
    print("="*70)
    print("COMPARAISON DES MODÈLES")
    print("="*70)
    print(f"Phrase: \"{text}\"")
    print("-"*70)
    
    models = [
        ("LSTM Simple", model_lstm_loaded if 'model_lstm_loaded' in globals() else None),
        ("BiLSTM + Attention", model_bilstm_att if 'model_bilstm_att' in globals() else None),
        ("CNN-BiLSTM + Attention", model_cnn_bilstm_att if 'model_cnn_bilstm_att' in globals() else None)
    ]
    
    for model_name, model in models:
        if model is None:
            print(f"\n{model_name}: Modèle non disponible")
            continue
            
        detected, all_probs = predict_emotions(text, model, tokenizer, threshold)
        
        print(f"\n{model_name}:")
        if detected:
            for emotion, prob in sorted(detected.items(), key=lambda x: x[1], reverse=True)[:3]:
                print(f"  - {emotion:20s}: {prob:.4f}")
        else:
            print(f"  Aucune émotion détectée (seuil >= {threshold})")
    
    print("="*70 + "\n")


# Test de comparaison
print("\n" + "="*70)
print("COMPARAISON DES 3 MODÈLES SUR DES PHRASES EXEMPLES")
print("="*70 + "\n")

comparison_phrases = [
    "I'm so grateful and happy!",
    "This makes me really angry and frustrated.",
    "I feel sad and disappointed about this situation."
]

for phrase in comparison_phrases:
    compare_models(phrase, threshold=0.3)

#  BERT-base pour la classification multi-label des émotions
- Utilise BertTokenizer (WordPiece, pas word-level)
Utilise BertTokenizer (WordPiece, pas word-level)
Ajoute automatiquement les tokens spéciaux [CLS] et [SEP]
Gère l'attention mask pour ignorer le padding
Max length: 128 tokens (suffisant pour les commentaires Reddit)
Comprend le contexte sémantique profond
Pré-entraîné sur des milliards de mots
Meilleure gestion des mots rares (subword tokenization)
Attention multi-têtes pour capturer différentes relations
Performance state-of-the-art sur NLP

In [82]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from transformers import TFBertModel
import tensorflow as tf

class BERTModel(Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        # Initialize layers in __init__
        self.bert = TFBertModel.from_pretrained(
            'bert-base-uncased',
            use_safetensors=False
        )
        self.dropout = Dropout(0.3)
        self.dense_1 = Dense(256, activation='relu')
        self.dense_2 = Dense(28, activation='sigmoid')
    
    def call(self, inputs, training=False):
        # inputs are REAL TensorFlow tensors (not KerasTensors)
        input_ids = inputs['input_ids']          # ← Real tensor
        attention_mask = inputs['attention_mask']  # ← Real tensor
        
        # Now BERT accepts the real tensors ✓
        output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            training=training
        )
        
        cls = output[1]
        x = self.dropout(cls, training=training)
        x = self.dense_1(x)
        x = self.dropout(x, training=training)
        x = self.dense_2(x)
        
        return x

# Create and use
model_bert = BERTModel()
# model.compile(optimizer='adam', loss='binary_crossentropy'

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [84]:

# custom_loss_function = create_custom_loss_function()
custom_loss_function = weighted_loss_factory(WEIGHTS_NUMPY)


# --- 5. COMPILATION DU MODÈLE ---
model_bert.compile(
    optimizer=Adam(learning_rate=2e-5),  # Learning rate faible pour fine-tuning
    loss=custom_loss_function,
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc', multi_label=True),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

print("\n✓ Modèle BERT compilé avec:")
print("  - Optimizer: Adam (lr=2e-5)")
print("  - Loss: Weighted Binary Cross-Entropy")
print("  - Metrics: Accuracy, AUC, Precision, Recall")


# --- 5. CALLBACKS POUR L'ENTRAÎNEMENT ---

callbacks_bert = [
    EarlyStopping(
        monitor='val_auc',
        patience=3,  # Patience réduite car BERT converge rapidement
        mode='max',
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        filepath='models/modele_bert_best.keras',
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-7,
        verbose=1
    )
]





✓ Modèle BERT compilé avec:
  - Optimizer: Adam (lr=2e-5)
  - Loss: Weighted Binary Cross-Entropy
  - Metrics: Accuracy, AUC, Precision, Recall


In [87]:
# --- 6. ENTRAÎNEMENT DU MODÈLE BERT (avec Subclassing API) ---

# --- 6.1 FONCTION DE TOKENIZATION ---
def encode_texts_for_bert(texts, tokenizer, max_length=128):
    """
    Tokenise les textes au format BERT.
    
    Args:
        texts: Liste de textes (strings)
        tokenizer: BertTokenizer
        max_length: Longueur maximale des séquences
    
    Returns:
        tuple: (input_ids, attention_mask) comme numpy arrays
    """
    input_ids = []
    attention_masks = []
    
    for text in texts:
        if not isinstance(text, str):
            text = str(text)
        
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors=None  # ← Important: Pas de tensors
        )
        
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    return np.array(input_ids), np.array(attention_masks)


# --- 6.2 PRÉPARER LES DONNÉES AVEC TOKENIZATION ---
print("Tokenisation des données d'entraînement...")
train_input_ids, train_attention_mask = encode_texts_for_bert(
    df_train_final['text'].tolist(),
    bert_tokenizer,
    MAX_LENGTH_BERT
)

print("Tokenisation des données de validation...")
dev_input_ids, dev_attention_mask = encode_texts_for_bert(
    df_dev_final['text'].tolist(),
    bert_tokenizer,
    MAX_LENGTH_BERT
)

print("Tokenisation des données de test...")
test_input_ids, test_attention_mask = encode_texts_for_bert(
    df_test_final['text'].tolist(),
    bert_tokenizer,
    MAX_LENGTH_BERT
)

print(f"✓ Tokenisation terminée")
print(f"  Train: input_ids {train_input_ids.shape}, attention_mask {train_attention_mask.shape}")
print(f"  Dev:   input_ids {dev_input_ids.shape}, attention_mask {dev_attention_mask.shape}")
print(f"  Test:  input_ids {test_input_ids.shape}, attention_mask {test_attention_mask.shape}")






Tokenisation des données d'entraînement...
Tokenisation des données de validation...
Tokenisation des données de test...
✓ Tokenisation terminée
  Train: input_ids (43410, 128), attention_mask (43410, 128)
  Dev:   input_ids (5426, 128), attention_mask (5426, 128)
  Test:  input_ids (5427, 128), attention_mask (5427, 128)


In [88]:

# --- 6.3 CRÉER LES DATASETS TENSORFLOW ---
def create_tf_dataset(input_ids, attention_mask, labels, batch_size=32, shuffle=True):
    """
    Crée un tf.data.Dataset à partir des données encodées.
    
    Args:
        input_ids: Array des IDs de tokens
        attention_mask: Array des masques d'attention
        labels: Array des labels multi-hot
        batch_size: Taille du batch
        shuffle: Mélanger les données
    
    Returns:
        tf.data.Dataset
    """
    # Créer le dataset à partir des arrays
    dataset = tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': input_ids,
            'attention_mask': attention_mask
        },
        labels
    ))
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(input_ids), reshuffle_each_iteration=True)
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    
    return dataset



In [89]:
df_train_final

,text,comment_id,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,My favourite food is anything I didn't have to...,eebbqej,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,"Now if he does off himself, everyone will thin...",ed00q6i,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,WHY THE FUCK IS BAYLESS ISOING,eezlygj,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,To make her feel threatened,ed7ypvh,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Dirty Southern Wankers,ed0bdzj,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43405,Added you mate well I’ve just got the bow and ...,edsb738,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
43406,Always thought that was funny but is it a refe...,ee7fdou,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
43407,What are you talking about? Anything bad that ...,efgbhks,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43408,"More like a baptism, with sexy results!",ed1naf8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
print("\nCréation des datasets TensorFlow...")
BATCH_SIZE = 16  # Réduit car BERT est gourmand en mémoire

# Extraire les labels depuis le DataFrame
emotion_columns = [col for col in df_train_final.columns if col != 'text' and col!='comment_id']
Y_train = df_train_final[emotion_columns].values.astype(np.float32)
Y_dev = df_dev_final[emotion_columns].values.astype(np.float32)
Y_test = df_test_final[emotion_columns].values.astype(np.float32)

train_dataset = create_tf_dataset(train_input_ids, train_attention_mask, Y_train, BATCH_SIZE)
dev_dataset = create_tf_dataset(dev_input_ids, dev_attention_mask, Y_dev, BATCH_SIZE, shuffle=False)
test_dataset = create_tf_dataset(test_input_ids, test_attention_mask, Y_test, BATCH_SIZE, shuffle=False)

print(f"✓ Datasets créés")
print(f"  Train: {len(train_dataset)} batches")
print(f"  Dev:   {len(dev_dataset)} batches")
print(f"  Test:  {len(test_dataset)} batches")


Création des datasets TensorFlow...
✓ Datasets créés
  Train: 2714 batches
  Dev:   340 batches
  Test:  340 batches


In [ ]:

# --- 6.5 ENTRAÎNEMENT ---
print("\n" + "="*70)
print("DÉBUT DE L'ENTRAÎNEMENT")
print("="*70)

history_bert = model_bert.fit(
    train_dataset,
    validation_data=dev_dataset,
    epochs=5,  # BERT converge rapidement, 3-5 epochs suffisent
    callbacks=callbacks_bert,
            class_weight=class_weights_dict,

    verbose=1
)

print("\n" + "="*70)
print("ENTRAÎNEMENT BERT TERMINÉ")
print("="*70)


# --- 7. SAUVEGARDE DU MODÈLE ---

print("\nSauvegarde du modèle...")

# Créer le répertoire s'il n'existe pas
import os
os.makedirs('models', exist_ok=True)

# Sauvegarder le modèle
chemin_sauvegarde_bert = 'models/modele_bert_final.keras'
model_bert.save(chemin_sauvegarde_bert)

print(f"✓ Modèle BERT sauvegardé dans: {chemin_sauvegarde_bert}")
print(f"✓ Meilleur modèle sauvegardé dans: models/modele_bert_best.keras")

# Sauvegarder le tokenizer
bert_tokenizer.save_pretrained('models/bert_tokenizer')
print(f"✓ Tokenizer BERT sauvegardé dans: models/bert_tokenizer")


# --- 8. ÉVALUATION SUR LE JEU DE TEST ---

print("\n" + "="*70)
print("ÉVALUATION DU MODÈLE BERT SUR LE JEU DE TEST")
print("="*70)

# Prédictions
print("\nGénération des prédictions...")
Y_pred_proba_bert = model_bert.predict(test_dataset, verbose=1)
Y_pred_binary_bert = (Y_pred_proba_bert > 0.5).astype(int)

print(f"✓ Prédictions générées")
print(f"  Probabilities shape: {Y_pred_proba_bert.shape}")
print(f"  Binary predictions shape: {Y_pred_binary_bert.shape}")

# Calcul des métriques
from sklearn.metrics import (
    hamming_loss, roc_auc_score, precision_score, recall_score, f1_score
)

h_loss_bert = hamming_loss(Y_test, Y_pred_binary_bert)
auc_roc_bert = roc_auc_score(Y_test, Y_pred_proba_bert, average='macro')
precision_micro_bert = precision_score(Y_test, Y_pred_binary_bert, average='micro', zero_division=0)
recall_micro_bert = recall_score(Y_test, Y_pred_binary_bert, average='micro', zero_division=0)
f1_micro_bert = f1_score(Y_test, Y_pred_binary_bert, average='micro', zero_division=0)
precision_macro_bert = precision_score(Y_test, Y_pred_binary_bert, average='macro', zero_division=0)
recall_macro_bert = recall_score(Y_test, Y_pred_binary_bert, average='macro', zero_division=0)
f1_macro_bert = f1_score(Y_test, Y_pred_binary_bert, average='macro', zero_division=0)

# Affichage des résultats
print("\nRÉSULTATS DU MODÈLE 4 (BERT-BASE)")
print("="*70)
print(f"Hamming Loss (H-Loss) : {h_loss_bert:.4f} (Proche de 0 : Mieux)")
print(f"AUC-ROC (Macro)       : {auc_roc_bert:.4f} (Proche de 1 : Mieux)")
print("-" * 35)
print("   Micro-Averaged (Global/Fréquent)")
print(f"   Precision (Micro)   : {precision_micro_bert:.4f}")
print(f"   Recall (Micro)      : {recall_micro_bert:.4f}")
print(f"   F1-score (Micro)    : {f1_micro_bert:.4f}")
print("-" * 35)
print("   Macro-Averaged (Classes Rares)")
print(f"   Precision (Macro)   : {precision_macro_bert:.4f}")
print(f"   Recall (Macro)      : {recall_macro_bert:.4f}")
print(f"   F1-score (Macro)    : {f1_macro_bert:.4f}")
print("="*70)


# --- 9. FONCTION DE PRÉDICTION POUR BERT ---

def predict_emotions_bert(text, threshold=0.5):
    """
    Prédit les émotions pour une phrase avec le modèle BERT.
    
    Args:
        text: La phrase à analyser (string)
        threshold: Seuil de décision
        
    Returns:
        dict: Émotions détectées et toutes les probabilités
    """
    # Tokenisation BERT
    encoded = bert_tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=MAX_LENGTH_BERT,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors=None  # ← Important pour la compatibilité
    )
    
    # Préparation des inputs
    input_ids = np.array([encoded['input_ids']])
    attention_mask = np.array([encoded['attention_mask']])
    
    # Prédiction avec le modèle Subclassing
    inputs = {
        'input_ids': input_ids,
        'attention_mask': attention_mask
    }
    probabilities = model_bert(inputs, training=False).numpy()[0]
    
    # Extraction des émotions
    detected_emotions = {}
    all_emotions = {}
    
    for i, emotion in enumerate(EMOTION_LABELS):
        prob = probabilities[i]
        all_emotions[emotion] = float(prob)
        if prob >= threshold:
            detected_emotions[emotion] = float(prob)
    
    return detected_emotions, all_emotions


# Test rapide du modèle BERT
print("\n" + "="*70)
print("TEST RAPIDE DU MODÈLE BERT")
print("="*70)

test_sentence = "I am so grateful and happy! This is amazing!"
detected, all_probs = predict_emotions_bert(test_sentence, threshold=0.3)

print(f"Phrase: \"{test_sentence}\"")
print("\nÉmotions détectées:")
for emotion, prob in sorted(detected.items(), key=lambda x: x[1], reverse=True):
    print(f"  - {emotion:20s}: {prob:.4f} ({prob*100:.2f}%)")

print("\nToutes les émotions et probabilités:")
for emotion, prob in sorted(all_probs.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"  - {emotion:20s}: {prob:.4f} ({prob*100:.2f}%)")

print("="*70)


# --- 10. SAUVEGARDE DES RÉSULTATS ---

# Sauvegarder les métriques
metrics_dict = {
    'model': 'BERT-Base',
    'hamming_loss': float(h_loss_bert),
    'auc_roc_macro': float(auc_roc_bert),
    'precision_micro': float(precision_micro_bert),
    'recall_micro': float(recall_micro_bert),
    'f1_micro': float(f1_micro_bert),
    'precision_macro': float(precision_macro_bert),
    'recall_macro': float(recall_macro_bert),
    'f1_macro': float(f1_macro_bert),
}

import json
with open('models/bert_metrics.json', 'w') as f:
    json.dump(metrics_dict, f, indent=4)

print(f"\n✓ Métriques sauvegardées dans: models/bert_metrics.json")


DÉBUT DE L'ENTRAÎNEMENT
Epoch 1/5
2588/2714 ━━━━━━━━━━━━━━━━━━━━ 9:34 5s/step - accuracy: 0.1624 - auc: 0.5007 - loss: 1.7011 - precision: 0.1028 - recall: 0.2150

In [ ]:
# FIXING
# --- 6. ENTRAÎNEMENT DU MODÈLE BERT ---

print("\n" + "="*70)
print("DÉBUT DE L'ENTRAÎNEMENT DU MODÈLE BERT")
print("="*70)
print("Note: BERT est plus lent à entraîner mais donne de meilleurs résultats")
print("      Comptez environ 5-10 minutes par epoch sur GPU, plus sur CPU")
print("="*70 + "\n")

# Préparer les inputs pour Keras
X_train_bert = [train_encodings['input_ids'], train_encodings['attention_mask']]
X_dev_bert = [dev_encodings['input_ids'], dev_encodings['attention_mask']]
X_test_bert = [test_encodings['input_ids'], test_encodings['attention_mask']]

history_bert = model_bert.fit(
    X_train_bert,
    Y_train,
    epochs=5,  # BERT converge rapidement, 3-5 epochs suffisent
    batch_size=16,  # Batch size réduit car BERT est très gourmand en mémoire
    validation_data=(X_dev_bert, Y_dev),
    callbacks=callbacks_bert,
        class_weight=class_weights_dict,

    verbose=1
)

print("\n" + "="*70)
print("ENTRAÎNEMENT BERT TERMINÉ")
print("="*70)


# --- 7. SAUVEGARDE DU MODÈLE ---

chemin_sauvegarde_bert = 'models/modele_bert_final.keras'
model_bert.save(chemin_sauvegarde_bert, save_format='keras')

print(f"\nModèle BERT sauvegardé dans: {chemin_sauvegarde_bert}")
print(f"Meilleur modèle sauvegardé dans: models/modele_bert_best.keras")


# --- 8. ÉVALUATION SUR LE JEU DE TEST ---

print("\n" + "="*70)
print("ÉVALUATION DU MODÈLE BERT SUR LE JEU DE TEST")
print("="*70)

# Prédictions
Y_pred_proba_bert = model_bert.predict(X_test_bert, batch_size=16, verbose=1)
Y_pred_binary_bert = (Y_pred_proba_bert > 0.5).astype(int)

# Calcul des métriques
h_loss_bert = hamming_loss(Y_test, Y_pred_binary_bert)
auc_roc_bert = roc_auc_score(Y_test, Y_pred_proba_bert, average='macro')
precision_micro_bert = precision_score(Y_test, Y_pred_binary_bert, average='micro', zero_division=0)
recall_micro_bert = recall_score(Y_test, Y_pred_binary_bert, average='micro', zero_division=0)
f1_micro_bert = f1_score(Y_test, Y_pred_binary_bert, average='micro', zero_division=0)
precision_macro_bert = precision_score(Y_test, Y_pred_binary_bert, average='macro', zero_division=0)
recall_macro_bert = recall_score(Y_test, Y_pred_binary_bert, average='macro', zero_division=0)
f1_macro_bert = f1_score(Y_test, Y_pred_binary_bert, average='macro', zero_division=0)

# Affichage des résultats
print("\nRÉSULTATS DU MODÈLE 4 (BERT-BASE)")
print("="*70)
print(f"Hamming Loss (H-Loss) : {h_loss_bert:.4f} (Proche de 0 : Mieux)")
print(f"AUC-ROC (Macro)       : {auc_roc_bert:.4f} (Proche de 1 : Mieux)")
print("-" * 35)
print("   Micro-Averaged (Global/Fréquent)")
print(f"   Precision (Micro)   : {precision_micro_bert:.4f}")
print(f"   Recall (Micro)      : {recall_micro_bert:.4f}")
print(f"   F1-score (Micro)    : {f1_micro_bert:.4f}")
print("-" * 35)
print("   Macro-Averaged (Classes Rares)")
print(f"   Precision (Macro)   : {precision_macro_bert:.4f}")
print(f"   Recall (Macro)      : {recall_macro_bert:.4f}")
print(f"   F1-score (Macro)    : {f1_macro_bert:.4f}")
print("="*70)


# --- 9. FONCTION DE PRÉDICTION POUR BERT ---

def predict_emotions_bert(text, threshold=0.5):
    """
    Prédit les émotions pour une phrase avec le modèle BERT.
    
    Args:
        text: La phrase à analyser (string)
        threshold: Seuil de décision
        
    Returns:
        dict: Émotions détectées et toutes les probabilités
    """
    # Tokenisation BERT
    encoded = bert_tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=MAX_LENGTH_BERT,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='np'
    )
    
    # Prédiction
    input_ids = encoded['input_ids']
    attention_mask = encoded['attention_mask']
    probabilities = model_bert.predict([input_ids, attention_mask], verbose=0)[0]
    
    # Extraction des émotions
    detected_emotions = {}
    all_emotions = {}
    
    for i, emotion in enumerate(EMOTION_LABELS):
        prob = probabilities[i]
        all_emotions[emotion] = prob
        if prob >= threshold:
            detected_emotions[emotion] = prob
    
    return detected_emotions, all_emotions


# Test rapide du modèle BERT
print("\n" + "="*70)
print("TEST RAPIDE DU MODÈLE BERT")
print("="*70)

test_sentence = "I am so grateful and happy! This is amazing!"
detected, all_probs = predict_emotions_bert(test_sentence, threshold=0.3)

print(f"Phrase: \"{test_sentence}\"")
print("\nÉmotions détectées:")
for emotion, prob in sorted(detected.items(), key=lambda x: x[1], reverse=True):
    print(f"  - {emotion:20s}: {prob:.4f} ({prob*100:.2f}%)")
print("="*70)


DÉBUT DE L'ENTRAÎNEMENT DU MODÈLE BERT
Note: BERT est plus lent à entraîner mais donne de meilleurs résultats
      Comptez environ 5-10 minutes par epoch sur GPU, plus sur CPU



NameError: name 'train_encodings' is not defined